In [1]:
import numpy as np
import cv2, os, pyautogui, time
from PIL import Image
import matplotlib.pyplot as plt
import warnings

from src.face_features import *
from src.tools import *

width, height= pyautogui.size()

In [21]:
fc.get_shape()

TensorShape([None, 320])

In [ ]:
from src.model import model

model.compile(optimizer='adam', loss="mse")
# model.summary() 

In [ ]:
# pickle.dump([X_left, X_right, X_face, X_mask, Y], open("save.pkl", "wb"))

# X_left = np.concatenate([X_left, X_left_old])
# X_right = np.concatenate([X_right, X_right_old])
# X_face = np.concatenate([X_face, X_face_old])
# X_mask = np.concatenate([X_mask, X_mask_old])
# Y = np.concatenate([Y, Y_old])

In [ ]:
X_left_old, X_right_old, X_face_old, X_mask_old, Y_old = X_left, X_right, X_face, X_mask, Y

In [5]:
import pickle

[X_left, X_right, X_face, X_mask, Y] = pickle.load(open("save.pkl", "rb"))
print(X_left.shape, X_right.shape, X_face.shape, X_mask.shape, Y.shape)

(9746, 64, 64, 1) (9746, 64, 64, 1) (9746, 64, 64, 1) (9746, 625) (9746, 2)


In [3]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(list(range(len(Y))), train_size=0.9, shuffle=True)

X_train = [X_left[train_set], X_right[train_set], X_face[train_set], X_mask[train_set]]
X_test = [X_left[test_set], X_right[test_set], X_face[test_set], X_mask[test_set]]

In [4]:
from keras.models import load_model
model = load_model("checkpoints")

In [22]:
history = model.fit(X_train, np.array(Y)[train_set], batch_size=256, epochs=200, validation_data=(X_test, np.array(Y)[test_set]))

Epoch 1/200
35/35 [==============================] - 8s 235ms/step - loss: 0.0472 - val_loss: 0.0375
Epoch 2/200
35/35 [==============================] - 7s 198ms/step - loss: 0.0353 - val_loss: 0.0342
Epoch 3/200
35/35 [==============================] - 7s 193ms/step - loss: 0.0313 - val_loss: 0.0283
Epoch 4/200
35/35 [==============================] - 7s 189ms/step - loss: 0.0264 - val_loss: 0.0238
Epoch 5/200
35/35 [==============================] - 7s 188ms/step - loss: 0.0224 - val_loss: 0.0236
Epoch 6/200
35/35 [==============================] - 7s 187ms/step - loss: 0.0176 - val_loss: 0.0168
Epoch 7/200
35/35 [==============================] - 7s 192ms/step - loss: 0.0134 - val_loss: 0.0123
Epoch 8/200
35/35 [==============================] - 7s 188ms/step - loss: 0.0103 - val_loss: 0.0088
Epoch 9/200
35/35 [==============================] - 7s 189ms/step - loss: 0.0081 - val_loss: 0.0082
Epoch 10/200
35/35 [==============================] - 7s 199ms/step - loss: 0.0073 - val_lo

KeyboardInterrupt: 

In [24]:
# Plot history: MAE
plt.figure(figsize=(10,6), dpi=80)
plt.plot(history.history['loss'], label='MSE (training data)')
plt.plot(history.history['val_loss'], label='MSE (validation data)')
plt.title('MSE over training')
plt.ylabel('MSE value')
plt.xlabel('No. epoch')
plt.legend(loc="upper left")
plt.show()

NameError: name 'history' is not defined

<Figure size 800x480 with 0 Axes>

In [ ]:
model.predict(X_train)

In [ ]:
model.save("checkpoints")